In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
import sys

# random sampling
from random import sample

# combination
from itertools import combinations

In [4]:
import warnings

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
random.seed(42)
_functional_relevance_lookup_dict = {}
_population = 1000
_generation = 500
_penalty_weight = 10
_num_one = 9
# _mu = 50
# _lambda = 100
_cxpb = 0.8
_mutpb = 0.003


# def trim_newline(s):
#     if s[-1] == '\n':
#         return s[0:-1]
    
#     return s

# workdir = "D://miR_Rank_Feautre_selection//GA_test"
# _functional_relevance_lookup_dict = {}
# with open(workdir + '//FR.txt', 'r') as fh:
#     fh.readline() # skip the first line
#     for line in fh:
#         elements = line.split('\t')
#         _functional_relevance_lookup_dict[elements[0]] = float(trim_newline(elements[1]))

# def functional_relevance_score(items):
#     sum_scores = 0
#     for x in items:
#         try:
#             sum_scores += _functional_relevance_lookup_dict[x]
#         except KeyError:
#             continue
    
#     return sum_scores

def average(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def evaluate(individual, X, y):
#     print (individual)
#     print ("Indivdial: num of 0: %d " % individual.count(0))
#     print ("Indivdial: num of 1: %d " % individual.count(1))
#     print ("The num of genes: %d" % len(individual))
    if(individual.count(0) != len(individual)):
        # 조건 1: 1의 개수가 5보다 클 때, penalty 값을 
        
        # get index with value 0
        cols = [i for i in range(len(individual)) if individual[i] == 1]

        
        #if len(individual) - len(cols) < 5:
        #    return 10000, 10000, 10000
        
        #if len(individual) - len(cols) > 20:
        #    return 10000, 10000, 10000
        
        # get features subset
#         X_parsed = X.drop(X.columns[cols], axis=1)
#         X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = RandomForestClassifier()
        #clf = LogisticRegression()
        
        # fr_score = functional_relevance_score(X_parsed.columns.to_list())
 
        avg_roc_score = 100 * average(cross_val_score(clf, X.iloc[:,cols], y, cv=3, scoring='roc_auc'))
        penalty_function = _penalty_weight * abs(individual.count(1) - 3)
#         print("Avg roc score: " + str(avg_roc_score) + " Penalty weight: " + str(penalty_function) + " len 1: " + str(individual.count(1)))
        # return average(cross_val_score(clf, X_subset, y, cv=5), scoring = "roc_auc") - _penalty_weight * abs(individual.count(1) - 3), 1/fr_score #, (len(individual) - len(cols)) / len(individual)
        return avg_roc_score - penalty_function,
#         shuffle = cross_validation.KFold(len(X), n_folds=3, shuffle=True)
#         scores = cross_val_score(rf, X, y, cv=shuffle, scoring='roc_auc')
    else:
        # return 10000, 10000 #, 10000
        return -100000,

In [6]:
workdir = "D://miR_Rank_Feautre_selection//Raw_data"
df_genomic = pd.read_csv(workdir + "//200617_95_samples_miRNA_HC_vs_T2DM_RPM.csv"
                    , index_col = ["id"])
y = df_genomic["Class"] #Class
#X_DN = df_e1.drop(["Class", 'hsa-let-7b-5p', 'hsa-miR-23b-3p', 'hsa-let-7c-5p', 'hsa-miR-30a-3p', 'hsa-miR-200c-3p'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-1290'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-642a-5p', 'hsa-miR-1343-3p', 'hsa-miR-642a-3p', 'hsa-miR-99b-3p', 'hsa-miR-671-3p', 'hsa-miR-3144-5p', 'hsa-miR-320c', 'hsa-miR-150-5p'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-642a-5p', 'hsa-miR-642a-3p', 'hsa-miR-99b-3p', 'hsa-miR-891a-5p', 'hsa-miR-320c', 'hsa-miR-150-5p'], axis = 1) #Rest dataseet
df_genomic = df_genomic.drop(["Class"], axis = 1) #Rest dataseet
# split data train 70 % and test 30 %
#drop and rotate dataset
df_genomic = df_genomic.T[(df_genomic != 0).any()].T

_number_of_samples = len(df_genomic)
_number_of_items = len(df_genomic.columns.to_list())


In [7]:
len(df_genomic)

95

In [8]:
# 각 miRNA에 대해 각 샘플들에서 가지고 있는 0의 총 비율이 0.9 이상이면 해당 miRNA 제거
cut_off = 0.9
for x in df_genomic.columns:
    try:
        num_zero = df_genomic[x].value_counts(0)[0]
        if num_zero / _number_of_samples > cut_off:
            df_genomic = df_genomic.drop([x], axis = 1)
    except KeyError:
        continue

In [9]:
df_genomic

,hsa-let-7a-3p,hsa-let-7a-5p,hsa-let-7b-3p,hsa-let-7b-5p,hsa-let-7c-5p,hsa-let-7d-3p,hsa-let-7d-5p,hsa-let-7e-3p,hsa-let-7e-5p,hsa-let-7f-1-3p,...,hsa-miR-95-3p,hsa-miR-95-5p,hsa-miR-9-5p,hsa-miR-96-5p,hsa-miR-98-3p,hsa-miR-98-5p,hsa-miR-99a-3p,hsa-miR-99a-5p,hsa-miR-99b-3p,hsa-miR-99b-5p
id,,,,,,,,,,,,,,,,,,,,,
SRR5034616,27.496209,5757.420584,20.354337,572.421088,23.211086,576.170571,606.523529,1.428375,76.060943,2.142562,...,0.178547,0.714187,0.000000,12.319730,16.783401,99.629123,0.357094,28.031850,2.856749,711.330509
SRR5034617,17.487052,12211.478810,13.349737,1326.064677,33.250223,324.103480,710.859702,1.158448,81.863676,2.661673,...,0.179284,0.013791,1.448060,58.529219,7.722988,173.808610,0.055164,11.763766,1.530807,399.830138
SRR5034618,51.480370,44465.243510,25.545184,2590.544879,240.095477,1057.931358,2228.671027,12.626341,1246.741466,4.972536,...,0.292502,0.000000,2.778770,16.233867,34.417748,617.471940,0.731255,14.478854,15.307610,2049.805991
SRR5034619,38.595591,10638.761160,27.037553,698.435723,32.610179,641.264713,901.939748,2.270329,147.364984,4.540658,...,1.031968,0.000000,4.953445,13.209186,21.052141,156.446300,0.206394,31.784604,2.063935,566.550254
SRR5034620,17.703098,3052.382241,18.228933,336.972342,13.671700,402.876946,283.950687,1.139308,41.365656,1.840421,...,0.087639,0.000000,0.525835,11.743640,9.727940,32.514106,0.262917,15.862677,0.964030,219.711226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR7412331,71.385132,78696.617310,30.201402,1902.688337,129.042355,135.906310,1290.423547,2.745582,675.413176,8.236746,...,4.118373,1.372791,164.734921,0.000000,6.863955,2315.898430,0.000000,10.982328,10.982328,404.973347
SRR7412332,62.178990,100443.756800,21.945526,1718.151784,138.988330,69.494165,929.027255,0.000000,429.766545,15.544747,...,1.828794,0.000000,43.891051,0.000000,1.828794,2008.015603,0.914397,51.206227,2.743191,629.105071
SRR7412333,66.274923,102931.718600,22.317066,1836.085884,135.254945,124.434550,1294.389828,0.000000,495.033100,18.935692,...,5.410198,1.352549,58.835901,0.676275,4.057648,2239.145622,0.000000,75.742769,4.057648,3494.311515


In [10]:

# # encode labels column to numbers
# le = LabelEncoder()
# #get label data by index and Encode items into 0 and 1
# le.fit(df_genomic.iloc[:, _labels.index(_target_label)])
# y = le.transform(df_genomic.iloc[:, _labels.index(_target_label)])
#get data excluding labels
X = df_genomic.iloc[:,]

#Min-Max normalization
# X = (X - X.min())/(X.max()-X.min())

# z-normalization
X = (X - X.mean())/X.std()

#X.to_csv('miR.txt', sep='\t', header=True)

In [11]:
creator.create("MyFitnessMulti", base.Fitness, weights=(1.0,)) # Note here <- I used only two weights!  (at first, I tried weights=(-1.0 , -1.0, 1.0)) but it crashes. With deap, you cannot do such a thing.
creator.create("Individual", list, fitness=creator.MyFitnessMulti)

toolbox = base.Toolbox()

#def f(a, b, c): 
#    return np.random.choice(np.arange(a, b), p=c)
  
# A partial function that calls f with 
# a as 3, b as 1 and c as 4. 
#g = partial(f, 0, 2) 




# Attribute generator
def random_individual_three():
    zero_list = [0 for i in range(len(X.columns))]
    index_for_one = np.random.choice(len(X.columns), _num_one, replace=True)
#     index_for_one = random.sample(range(len(X.columns)),_num_one) # 중복을 허용하지 않는 index random sampling
    # index_for_one = list(combinations(range(len(X.columns)),_num_one)) # combination 생성
    for i in index_for_one:
        zero_list[i]= 1 
    
#     print(zero_list.count(1))
    return zero_list
    

# Structure initializers
toolbox.register("rand_ind",random_individual_three)
toolbox.register("individual", tools.initIterate, creator.Individual
                 ,toolbox.rand_ind) #, n=30) #
# toolbox.register("individual", tools.initIterate, creator.Individual, random.sample(range(len(X.columns)),_num_one))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registration of crossover & mutation functions
#toolbox.register("mate", tools.cxUniform, indpb=0.8)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=_mutpb)

# Registration of selection function
toolbox.register("select", tools.selBest)
# toolbox.register("select", tools.selRoulette)
# toolbox.register("select", tools.selNSGA2) # NSGA-2 applies to multi-objective problems such as knapsack problem
# toolbox.register("select", tools.selTournament, k=1, tournsize=3)

toolbox.register("evaluate", evaluate, X=X, y=y)
logbook = tools.Logbook()

In [12]:
len(X.columns)

1169

In [13]:
# from functools import partial
# random.seed(42)
# gen_idx = random.sample(range(len(X.columns)), _num_one)
# print(gen_idx)
# print(list(gen_idx))

In [14]:
pop = toolbox.population(n=_population)
hof = tools.HallOfFame(1) # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution
# hof = tools.ParetoFront() # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

# pop, log = algorithms.eaMuPlusLambda(pop, toolbox, _mu, _lambda, _cxpb, _mutpb, 
#                                      ngen=_generation, stats=stats, halloffame=hof) #, verbose=True)

# pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=_generation, 
#                                   stats=stats, halloffame=hof, verbose=True)

population = []

for i in range(0,_generation + 1):
    print ("Generation: %d" % i)
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=1, stats=stats, halloffame=hof, verbose=True)
    population.append(pop)
# pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=_generation, stats=stats, halloffame=hof, verbose=True)

#for ind, fit in zip (pop, )

Generation: 0
gen	nevals	avg         	std         	min           	max          
0  	1000  	[-5.5039931]	[9.42612736]	[-34.89128805]	[22.32118758]
1  	779   	[-5.80607812]	[17.6652916]	[-75.7917229] 	[57.10676263]
Generation: 1
gen	nevals	avg          	std         	min          	max          
0  	0     	[-5.80607812]	[17.6652916]	[-75.7917229]	[57.10676263]
1  	770   	[-5.76354851]	[22.44263908]	[-93.73369321]	[59.89803569]
Generation: 2
gen	nevals	avg          	std          	min           	max          
0  	0     	[-5.76354851]	[22.44263908]	[-93.73369321]	[59.89803569]
1  	796   	[-5.9184233] 	[26.09676512]	[-135.03073924]	[58.91962813]
Generation: 3
gen	nevals	avg         	std          	min            	max          
0  	0     	[-5.9184233]	[26.09676512]	[-135.03073924]	[58.91962813]
1  	797   	[-5.90134503]	[29.04677997]	[-141.16209327]	[67.65032239]
Generation: 4
gen	nevals	avg          	std          	min            	max          
0  	0     	[-5.90134503]	[29.04677997]	[-141.1620932

1  	774   	[-1512.2325626] 	[12153.9108383] 	[-100000.]	[77.53411306]
Generation: 38
gen	nevals	avg            	std            	min       	max          
0  	0     	[-1512.2325626]	[12153.9108383]	[-100000.]	[77.53411306]
1  	792   	[-1812.56073399]	[13293.57471069]	[-100000.]	[83.10466337]
Generation: 39
gen	nevals	avg             	std             	min       	max          
0  	0     	[-1812.56073399]	[13293.57471069]	[-100000.]	[83.10466337]
1  	842   	[-1812.4911396] 	[13293.58790753]	[-100000.]	[77.86024891]
Generation: 40
gen	nevals	avg            	std             	min       	max          
0  	0     	[-1812.4911396]	[13293.58790753]	[-100000.]	[77.86024891]
1  	774   	[-1912.24848328]	[13650.93412059]	[-100000.]	[79.56590193]
Generation: 41
gen	nevals	avg             	std             	min       	max          
0  	0     	[-1912.24848328]	[13650.93412059]	[-100000.]	[79.56590193]
1  	772   	[-2012.79092368]	[13998.3368093] 	[-100000.]	[82.3174389] 
Generation: 42
gen	nevals	avg       

1  	796   	[-6119.55741341]	[23928.18612872]	[-100000.]	[80.96041386]
Generation: 75
gen	nevals	avg             	std             	min       	max          
0  	0     	[-6119.55741341]	[23928.18612872]	[-100000.]	[80.96041386]
1  	831   	[-6219.39138926]	[24110.70946081]	[-100000.]	[80.76698156]
Generation: 76
gen	nevals	avg             	std             	min       	max          
0  	0     	[-6219.39138926]	[24110.70946081]	[-100000.]	[80.76698156]
1  	822   	[-6519.93528565]	[24647.46746595]	[-100000.]	[80.96791123]
Generation: 77
gen	nevals	avg             	std             	min       	max          
0  	0     	[-6519.93528565]	[24647.46746595]	[-100000.]	[80.96791123]
1  	800   	[-6720.26428025]	[24996.90513748]	[-100000.]	[81.67266457]
Generation: 78
gen	nevals	avg             	std             	min       	max          
0  	0     	[-6720.26428025]	[24996.90513748]	[-100000.]	[81.67266457]
1  	816   	[-6720.35360249]	[24996.88122378]	[-100000.]	[81.66666667]
Generation: 79
gen	nevals	avg 

1  	756   	[-10526.50539586]	[30646.40447422]	[-100000.]	[81.69890538]
Generation: 112
gen	nevals	avg              	std             	min       	max          
0  	0     	[-10526.50539586]	[30646.40447422]	[-100000.]	[81.69890538]
1  	812   	[-11027.03139151]	[31279.61165551]	[-100000.]	[83.64447443]
Generation: 113
gen	nevals	avg              	std             	min       	max          
0  	0     	[-11027.03139151]	[31279.61165551]	[-100000.]	[83.64447443]
1  	800   	[-11527.57441445]	[31892.39202638]	[-100000.]	[85.78872395]
Generation: 114
gen	nevals	avg              	std             	min       	max          
0  	0     	[-11527.57441445]	[31892.39202638]	[-100000.]	[85.78872395]
1  	796   	[-11627.72862198]	[32012.58679717]	[-100000.]	[81.59769081]
Generation: 115
gen	nevals	avg              	std             	min       	max          
0  	0     	[-11627.72862198]	[32012.58679717]	[-100000.]	[81.59769081]
1  	792   	[-11827.72991378]	[32250.80897471]	[-100000.]	[82.24246514]
Generation: 1

1  	749   	[-14634.06484555]	[35296.69497033]	[-100000.]	[80.63052931]
Generation: 148
gen	nevals	avg              	std             	min       	max          
0  	0     	[-14634.06484555]	[35296.69497033]	[-100000.]	[80.63052931]
1  	766   	[-14534.25963488]	[35196.13530209]	[-100000.]	[80.63052931]
Generation: 149
gen	nevals	avg              	std             	min       	max          
0  	0     	[-14534.25963488]	[35196.13530209]	[-100000.]	[80.63052931]
1  	818   	[-14734.2238514] 	[35396.5433859] 	[-100000.]	[82.19748088]
Generation: 150
gen	nevals	avg             	std            	min       	max          
0  	0     	[-14734.2238514]	[35396.5433859]	[-100000.]	[82.19748088]
1  	807   	[-14734.23572874]	[35396.53791262]	[-100000.]	[82.96221323]
Generation: 151
gen	nevals	avg              	std             	min       	max          
0  	0     	[-14734.23572874]	[35396.53791262]	[-100000.]	[82.96221323]
1  	832   	[-15034.40614635]	[35692.84501313]	[-100000.]	[84.59289249]
Generation: 152
g

...

1  	782   	[-24258.46996101]	[42796.61839327]	[-100000.]	[75.94092068]
Generation: 328
gen	nevals	avg              	std             	min       	max          
0  	0     	[-24258.46996101]	[42796.61839327]	[-100000.]	[75.94092068]
1  	814   	[-24458.68889339]	[42916.15830552]	[-100000.]	[78.7037037] 
Generation: 329
gen	nevals	avg              	std             	min       	max         
0  	0     	[-24458.68889339]	[42916.15830552]	[-100000.]	[78.7037037]
1  	811   	[-24458.71829885]	[42916.14298592]	[-100000.]	[83.30334383]
Generation: 330
gen	nevals	avg              	std             	min       	max          
0  	0     	[-24458.71829885]	[42916.14298592]	[-100000.]	[83.30334383]
1  	808   	[-24659.25373819]	[43034.23729941]	[-100000.]	[83.30334383]
Generation: 331
gen	nevals	avg              	std             	min       	max          
0  	0     	[-24659.25373819]	[43034.23729941]	[-100000.]	[83.30334383]
1  	766   	[-24659.16625356]	[43034.2890636] 	[-100000.]	[84.45044235]
Generation: 332

1  	780   	[-26263.10454566]	[43934.89814197]	[-100000.]	[79.58464537]
Generation: 364
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26263.10454566]	[43934.89814197]	[-100000.]	[79.58464537]
1  	782   	[-26163.20709927]	[43880.66387157]	[-100000.]	[81.58644474]
Generation: 365
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26163.20709927]	[43880.66387157]	[-100000.]	[81.58644474]
1  	802   	[-26163.38321637]	[43880.56079571]	[-100000.]	[75.34487929]
Generation: 366
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26163.38321637]	[43880.56079571]	[-100000.]	[75.34487929]
1  	769   	[-26063.69978558]	[43825.90321126]	[-100000.]	[79.53216374]
Generation: 367
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26063.69978558]	[43825.90321126]	[-100000.]	[79.53216374]
1  	810   	[-26063.64861224]	[43825.9351492] 	[-100000.]	[80.33063428]
Generation: 3

1  	820   	[-26467.40237742]	[44039.81150789]	[-100000.]	[85.0239916] 
Generation: 400
gen	nevals	avg              	std             	min       	max         
0  	0     	[-26467.40237742]	[44039.81150789]	[-100000.]	[85.0239916]
1  	760   	[-26567.61517019]	[44092.97488848]	[-100000.]	[85.0239916]
Generation: 401
gen	nevals	avg              	std             	min       	max         
0  	0     	[-26567.61517019]	[44092.97488848]	[-100000.]	[85.0239916]
1  	791   	[-26367.48939871]	[43986.18053981]	[-100000.]	[85.0239916]
Generation: 402
gen	nevals	avg              	std             	min       	max         
0  	0     	[-26367.48939871]	[43986.18053981]	[-100000.]	[85.0239916]
1  	794   	[-26367.44326961]	[43986.21065623]	[-100000.]	[88.35282651]
Generation: 403
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26367.44326961]	[43986.21065623]	[-100000.]	[88.35282651]
1  	795   	[-26267.75332734]	[43932.15047372]	[-100000.]	[83.57324936]
Generation: 404
gen	n

1  	830   	[-26872.53466112]	[44248.1463384] 	[-100000.]	[85.45134203]
Generation: 436
gen	nevals	avg              	std            	min       	max          
0  	0     	[-26872.53466112]	[44248.1463384]	[-100000.]	[85.45134203]
1  	846   	[-27072.72768406]	[44351.99181379]	[-100000.]	[86.31354026]
Generation: 437
gen	nevals	avg              	std             	min       	max          
0  	0     	[-27072.72768406]	[44351.99181379]	[-100000.]	[86.31354026]
1  	784   	[-27072.65179787]	[44352.039923]  	[-100000.]	[86.77462888]
Generation: 438
gen	nevals	avg              	std           	min       	max          
0  	0     	[-27072.65179787]	[44352.039923]	[-100000.]	[86.77462888]
1  	821   	[-26973.0217169] 	[44299.97959143]	[-100000.]	[80.89293747]
Generation: 439
gen	nevals	avg             	std             	min       	max          
0  	0     	[-26973.0217169]	[44299.97959143]	[-100000.]	[80.89293747]
1  	842   	[-26872.92521068]	[44247.91653089]	[-100000.]	[81.39901035]
Generation: 440
gen	n

1  	816   	[-26175.79130904]	[43873.25388822]	[-100000.]	[82.16749138]
Generation: 473
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26175.79130904]	[43873.25388822]	[-100000.]	[82.16749138]
1  	801   	[-26176.06863773]	[43873.08711298]	[-100000.]	[82.16749138]
Generation: 474
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26176.06863773]	[43873.08711298]	[-100000.]	[82.16749138]
1  	830   	[-26376.55086445]	[43980.81163942]	[-100000.]	[82.16749138]
Generation: 475
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26376.55086445]	[43980.81163942]	[-100000.]	[82.16749138]
1  	822   	[-26276.58957415]	[43926.93035222]	[-100000.]	[88.37531864]
Generation: 476
gen	nevals	avg              	std             	min       	max          
0  	0     	[-26276.58957415]	[43926.93035222]	[-100000.]	[88.37531864]
1  	794   	[-26376.74307692]	[43980.70122863]	[-100000.]	[85.34263008]
Generation: 4

In [15]:
print("Individulas in the hall of fame population")
for x in hof:
    print(x)

Individulas in the hall of fame population
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
print("The # of individuals in the hof population: %d " % len(hof))

The # of individuals in the hof population: 1 


In [17]:
# Best individual 선택
#min_error_rate = 1.0
#for i in hof:
#    if (i.fitness.values[0] < min_error_rate):
#        min_error_rate = i.fitness.values[0]
#        _individual = i
# _individual = tools.selRoulette(hof, 1)[0]
_individual = tools.selBest(hof, 1)[0]
# _individual = tools.selNSGA2(hof, 1)[0]
_individual_features = [X.columns[i] for i in range(len(_individual)) if _individual[i] == 1]
print("The best individual is :" + str(_individual))

The best individual is :[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [18]:
print("The best features are: \t" + str(_individual_features))

The best features are: 	['hsa-let-7b-5p', 'hsa-miR-125b-5p', 'hsa-miR-7-5p']


In [19]:
#min_error_rate = 1.0
#for i in hof:
#    if (i.fitness.values[0] < min_error_rate):
#        min_error_rate = i.fitness.values[0]
#        _individual = i

#_individual_features = [X.columns[i] for i in range(len(_individual)) if _individual[i] == 1]
print('Fitness score: \t' + str(_individual.fitness))



Fitness score: 	(92.89248762932974,)


In [20]:
print('Number of Features in Subset: \t' + str(_individual.count(1)))


Number of Features in Subset: 	3


In [21]:
print('Individual: \t' + str(_individual))

Individual: 	[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
print('Feature Subset\t: ' + str(_individual_features))

Feature Subset	: ['hsa-let-7b-5p', 'hsa-miR-125b-5p', 'hsa-miR-7-5p']


In [23]:
_individual.fitness

deap.creator.MyFitnessMulti((92.89248762932974,))

In [24]:
len(_individual_features)

3

In [25]:
for x in _individual_features:
    print ("'{0}', ".format(x), end='')

'hsa-let-7b-5p', 'hsa-miR-125b-5p', 'hsa-miR-7-5p', 